In [1]:
import pandas as pd
import numpy as np

In [2]:
# needed to merge together driving and transit estimates manually
zip_estimates = pd.read_csv('data-outputs/zip_summary_driving.csv')

In [3]:
surveys = pd.read_excel('panynj/all_years.xlsx',sheetname='data')

In [4]:
surveys.columns

Index(['Year', 'WEIGHT', 'Language', 'TOTAL', 'AIRPORT', 'TERMINAL',
       'CONCOURSE', 'GATE', 'FLIGHTTYPE', 'AirportNovice', 'TerminalNovice',
       'AIRLINE', 'DESTINATION', 'DestinationCountry', 'CheckinLocation',
       'PartySize', 'KidsInParty', 'NumKids', 'GENDER', 'Age', 'AgeGROUPS',
       'HHIncome', 'ODTravelTimeToAirport', 'AirportDwellTimeDepartOD',
       'PostSecChkDwellTimeOD', 'BagsCheckedSinglePass',
       'BagsChecked2PlusPass', 'NumBagsChecked', 'CheckedBagIncidence',
       'TripPurpose', 'TripOriginOD', 'TripOriginLocationOD', 'TOManhattan',
       'TONYC', 'TONY', 'TONJ', 'TOCT', 'TOPA', 'TOOtherUS', 'TOZip',
       'TOTownCity', 'VisitorVsResident', 'NightsVisited', 'NightsAway',
       'ResidentialRegion', 'PrimaryResidentialLocation', 'MAINMODE',
       'SunnyWeather', 'DryWeather', 'WetWeather', 'DryOrWet', 'ZipProvided'],
      dtype='object')

In [5]:
surveys.TOZip[surveys.TOZip!=' '].shape

(11447,)

In [6]:
# label each value of MAINMODE with a category
modes_reported = {'A':'train',
'E':'train',
'J':'train',
'Z':'train',
'LIRR Connecting to JFK AirTrain':'train',
'Other (NYC Subway, including Connection to JFK AirTrain)':'train',
'Airlink New York':'van',
'All County Express':'van',
'Connecticut Limousine':'van',
"Dave's Best Limousine":'van',
'ETS Airport Shuttle':'van',
'Other (Shared-Ride Van/Service)':'van',
'Prime Time Shuttle of Connecticut':'van',
'State Shuttle':'van',
'SuperShuttle from Long Island':'van',
'SuperShuttle from Manhattan':'van',
'Limo/Executive Car/Town Car Service':'limo',
'Chartered/Tour Bus':'bus',
'NYC Airporter Bus from JFK/LaGuardia Airports':'bus',
'NYC Airporter Bus from Manhattan':'bus',
'Public/City Bus (that is, a local bus)':'bus',
'Transbridge Bus':'bus',
'Taxi':'taxi',
'Air Park':'local_van',
'Dollar Airport Parking':'local_van',
'Hilton Hotel':'local_van',
'Holiday Inn FSP':'local_van', 
'Hotel/Motel Shuttle/Van':'local_van',
'JFK Marriott Hotel':'local_van',
'National Airport Parking':'local_van',
'Other (Off-Airport Parking Shuttle/Van)':'local_van',
'Park Plus':'local_van',
'Sky Park/Central Parking':'local_van',
'The Parking Spot JFK/LGA':'local_van',
'U-Save':'local_van',
'Vista Airport Parking':'local_van',
'Drove Your Own Car':'private',
'Passenger in Car Parked at Airport':'private',
'Passenger in Car and Dropped Off at Airport':'private',
'Avis':'rental',
'Budget':'rental',
'Dollar':'rental',
'Enterprise':'rental',
'Hertz':'rental',
'National':'rental',
'Other (Drove Rental Car)':'rental'}

In [7]:
# label each value of MAINMODE with a category
modes_model = {'A':'transit',
'E':'transit',
'J':'transit',
'Z':'transit',
'LIRR Connecting to JFK AirTrain':'transit',
'Other (NYC Subway, including Connection to JFK AirTrain)':'transit',
'Airlink New York':'other',
'All County Express':'other',
'Connecticut Limousine':'other',
"Dave's Best Limousine":'other',
'ETS Airport Shuttle':'other',
'Other (Shared-Ride Van/Service)':'other',
'Prime Time Shuttle of Connecticut':'other',
'State Shuttle':'other',
'SuperShuttle from Long Island':'other',
'SuperShuttle from Manhattan':'other',
'Limo/Executive Car/Town Car Service':'taxi-FHV',
'Chartered/Tour Bus':'other',
'NYC Airporter Bus from JFK/LaGuardia Airports':'other',
'NYC Airporter Bus from Manhattan':'other',
'Public/City Bus (that is, a local bus)':'transit',
'Transbridge Bus':'other',
'Taxi':'taxi-FHV',
'Air Park':'other',
'Dollar Airport Parking':'other',
'Hilton Hotel':'other_free',
'Holiday Inn FSP':'other_free', 
'Hotel/Motel Shuttle/Van':'other_free',
'JFK Marriott Hotel':'other_free',
'National Airport Parking':'other',
'Other (Off-Airport Parking Shuttle/Van)':'other',
'Park Plus':'other',
'Sky Park/Central Parking':'other',
'The Parking Spot JFK/LGA':'other',
'U-Save':'other',
'Vista Airport Parking':'other',
'Drove Your Own Car':'other',
'Passenger in Car Parked at Airport':'other',
'Passenger in Car and Dropped Off at Airport':'other_free',
'Avis':'other',
'Budget':'other',
'Dollar':'other',
'Enterprise':'other',
'Hertz':'other',
'National':'other',
'Other (Drove Rental Car)':'other',
'Alamo':'other',
'AviStar/AviStar Haynes':'other',
'Corporate':'taxi-FHV',
"Don't know":'other',
'Four Points Sheraton Hotel Parking':'other',
'JFK AirTrain':'transit',
'JFK Crowne Plaza':'other_free',
'Metro-North Railroad':'transit',
'Metro-North Railroad Hudson Line Connecting to a Bus/Taxi':'transit',
"Mr. B's Park & Lock":'other',
"NJ Transit or Amtrak Connecting to Newark Liberty Airport AirTrain":'transit',
"NJ Transit or Amtrak Connecting to Shuttle Bus to Newark Liberty Airport":'transit',
"NY Airport Service Bus from Manhattan":'other',
"NY Airport Service Bus to JFK/LaGuardia Airports":'other',
"NYC Airporter Bus to JFK/LaGuardia Airports":'other',
"Newark Liberty Airport Express Bus from Manhattan":'other',
'None/Nothing/NA':'other',
"Olympia Trails":'other',
'On-Airport Terminal Access Shuttle Bus':'other_free',
'Other':'other',
'Other (Please Specify)':'other',
'Other Specify':'other',
'Payless':'other',
'Refused':'other',
'Smart Park':'other',
'SuperShuttle':'other',
'SuperShuttle to Long Island':'other',
'SuperShuttle to Manhattan':'other',
'SuperShuttle to Manhattan/Long Island':'other',
'The Parking Spot Haynes':'other',
'Visconti Ground Transportation':'other',
'Westchester Express':'other'}

In [8]:
# check that all possible MAINMODE values are assigned a grouping
mode_compare = pd.DataFrame(surveys.groupby('MAINMODE').size(),columns=['count'])
mode_compare['2014_reported_mode'] = pd.Series(modes_reported)
mode_compare['MNL_model_mode'] = pd.Series(modes_model)
mode_codes = {'taxi-FHV':1,'transit':2,'other':3,'other_free':4}
mode_compare = mode_compare.merge(pd.DataFrame.from_dict(mode_codes,orient='index').rename(columns={0:'CHOICE'}),left_on='MNL_model_mode',right_index=True)
mode_compare

,count,2014_reported_mode,MNL_model_mode,CHOICE
MAINMODE,,,,
A,359,train,transit,2
E,419,train,transit,2
J,34,train,transit,2
JFK AirTrain,1,NaN,transit,2
LIRR Connecting to JFK AirTrain,509,train,transit,2
Metro-North Railroad,1,NaN,transit,2
Metro-North Railroad Hudson Line Connecting to a Bus/Taxi,2,NaN,transit,2
NJ Transit or Amtrak Connecting to Newark Liberty Airport AirTrain,506,NaN,transit,2
NJ Transit or Amtrak Connecting to Shuttle Bus to Newark Liberty Airport,78,NaN,transit,2


In [12]:
# extract only useful survey data
df = surveys.query('AIRPORT=="JFK"')
df = df[['TOZip','MAINMODE','FLIGHTTYPE']]
# also convert zip
df['zipcode'] = pd.to_numeric(df['TOZip'],errors='coerce')

In [13]:
df.groupby('FLIGHTTYPE').size()

FLIGHTTYPE
Domestic         4870
International    7076
dtype: int64

In [17]:
df['INTL'] = pd.get_dummies(df['FLIGHTTYPE'])['International']

In [19]:
joined = df.dropna(how='any',subset=['zipcode']).merge(zip_estimates,on='zipcode',how='inner')

In [20]:
joined = joined.merge(mode_compare,left_on='MAINMODE',right_index=True)

In [21]:
joined['taxi_mean_duration'] = np.divide(joined['taxi_mean_duration'],60)
joined['transit_duration'] = np.divide(joined['transit_duration'],60)
joined['duration_in_traffic'] = np.divide(joined['duration_in_traffic'],60)
joined['driving_duration'] = np.divide(joined['driving_duration'],60)

In [22]:
joined.head()

,TOZip,MAINMODE,FLIGHTTYPE,zipcode,INTL,taxi_mean_pickup_longitude,taxi_mean_pickup_latitude,taxi_mean_trip_distance,taxi_mean_duration,taxi_mean_total_ex_tip,...,taxi_N,route,driving_distance,duration_in_traffic,driving_duration,transit_duration,count,2014_reported_mode,MNL_model_mode,CHOICE
0,10019,Limo/Executive Car/Town Car Service,Domestic,10019,0,-73.981769,40.763671,17.908526,47.879023,56.067815,...,33816,I-495 E and I-678 S,18.362756,49.866667,43.016667,58.083333,2736,limo,taxi-FHV,1
9,10019,Limo/Executive Car/Town Car Service,International,10019,1,-73.981769,40.763671,17.908526,47.879023,56.067815,...,33816,I-495 E and I-678 S,18.362756,49.866667,43.016667,58.083333,2736,limo,taxi-FHV,1
10,10019,Limo/Executive Car/Town Car Service,International,10019,1,-73.981769,40.763671,17.908526,47.879023,56.067815,...,33816,I-495 E and I-678 S,18.362756,49.866667,43.016667,58.083333,2736,limo,taxi-FHV,1
13,10019,Limo/Executive Car/Town Car Service,International,10019,1,-73.981769,40.763671,17.908526,47.879023,56.067815,...,33816,I-495 E and I-678 S,18.362756,49.866667,43.016667,58.083333,2736,limo,taxi-FHV,1
23,10019,Limo/Executive Car/Town Car Service,Domestic,10019,0,-73.981769,40.763671,17.908526,47.879023,56.067815,...,33816,I-495 E and I-678 S,18.362756,49.866667,43.016667,58.083333,2736,limo,taxi-FHV,1


In [23]:
biogeme_data = joined.loc[:,['CHOICE','taxi_mean_duration','taxi_mean_total_w_inferred_tip','transit_duration','INTL']].reset_index(drop=True)
biogeme_data['TRANSIT_COST'] = 7.75
biogeme_data['FREE_COST'] = 0

In [24]:
biogeme_data['av1'],biogeme_data['av2'],biogeme_data['av3'] = 1,1,1
biogeme_data['av4'] = 0
biogeme_data.loc[biogeme_data['CHOICE']==4,'av4'] = 1

In [25]:
biogeme_data.rename(columns={'taxi_mean_duration':'TAXI_TT','taxi_mean_total_w_inferred_tip':'TAXI_COST','transit_duration':'TRANSIT_TT'},inplace=True)

In [26]:
biogeme_data

,CHOICE,TAXI_TT,TAXI_COST,TRANSIT_TT,INTL,TRANSIT_COST,FREE_COST,av1,av2,av3,av4
0,1,47.879023,66.695924,58.083333,0,7.75,0,1,1,1,0
1,1,47.879023,66.695924,58.083333,1,7.75,0,1,1,1,0
2,1,47.879023,66.695924,58.083333,1,7.75,0,1,1,1,0
3,1,47.879023,66.695924,58.083333,1,7.75,0,1,1,1,0
4,1,47.879023,66.695924,58.083333,0,7.75,0,1,1,1,0
5,1,47.879023,66.695924,58.083333,1,7.75,0,1,1,1,0
6,1,47.879023,66.695924,58.083333,0,7.75,0,1,1,1,0
7,1,9.979688,44.238117,31.466667,0,7.75,0,1,1,1,0
8,1,35.056290,68.523287,72.216667,1,7.75,0,1,1,1,0
9,1,35.056290,68.523287,72.216667,0,7.75,0,1,1,1,0


In [29]:
biogeme_data['TRANSIT_TT'] = biogeme_data['TRANSIT_TT'].fillna(999.99)
biogeme_data.to_csv('models-data/data_27jun.dat',sep='\t',index_label='ID')